In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
import talib as ta

from sklearn import svm
from collections import Counter
from sklearn.preprocessing import StandardScaler

In [2]:
df = pd.read_csv("WTI.csv")
df = df.drop(df.columns[[0]], axis=1)
df.head()
df.shape

(2605, 4)

In [3]:
N = 2
w = 2*N + 1

for i in range(0,df.shape[0]-1):
    df.loc[df.index[i],'c'] = df['Uclose'][i] - df['Uclose'][i+1]
    
df.head(10)

,Uopen,Ulow,Uhigh,Uclose,c
0,52.15,51.93,53.25,52.20,-0.14
1,52.65,52.04,53.58,52.34,-0.51
2,52.76,51.85,53.30,52.85,0.24
3,52.91,52.29,53.25,52.61,-0.16
4,52.17,51.82,52.95,52.77,0.50
5,53.10,51.44,53.16,52.27,-0.86
6,52.93,52.75,53.41,53.13,-0.11
7,53.13,53.05,53.79,53.24,0.26
8,52.00,51.76,53.13,52.98,0.62
9,52.20,51.83,52.61,52.36,0.16


In [4]:
for i in range(0,df.shape[0]-(2*N+1)):
    s = 0
    for j in range((2*N)+1):
        s += df['Uclose'][i+j]
    s = s*1.0
    s = s/((2*N)+1)
    df.loc[df.index[i+N],'ma'] = s

df.head()

,Uopen,Ulow,Uhigh,Uclose,c,ma
0,52.15,51.93,53.25,52.20,-0.14,NaN
1,52.65,52.04,53.58,52.34,-0.51,NaN
2,52.76,51.85,53.30,52.85,0.24,52.554
3,52.91,52.29,53.25,52.61,-0.16,52.568
4,52.17,51.82,52.95,52.77,0.50,52.726


In [5]:
for i in range(0,df.shape[0]-N):
    h = df['Uhigh'][i]
    l = df['Ulow'][i]
    for j in range(N):
        if h<df['Uhigh'][i+j]:
            h = df['Uhigh'][i+j]
        if l>df['Ulow'][i+j]:
            l = df['Ulow'][i+j]
    R = (h - df['Uclose'][i+N])*(-100)/(h-l)
    df.loc[df.index[i+N],'R'] = R
    

df.head()

,Uopen,Ulow,Uhigh,Uclose,c,ma,R
0,52.15,51.93,53.25,52.20,-0.14,NaN,NaN
1,52.65,52.04,53.58,52.34,-0.51,NaN,NaN
2,52.76,51.85,53.30,52.85,0.24,52.554,-44.242424
3,52.91,52.29,53.25,52.61,-0.16,52.568,-56.069364
4,52.17,51.82,52.95,52.77,0.50,52.726,-36.551724


In [6]:
for i in range(0,df.shape[0]-N):
    h = 0
    l = 0
    vr = 0
    for j in range(N):
        h += df['Uhigh'][i+j]
        l += df['Ulow'][i+j]
    h = h*1.0/N
    l = l*1.0/N
    x = (df['Uhigh'][i+N] + df['Uclose'][i+N-1])/2
    y = (df['Ulow'][i+N] + df['Uclose'][i+N-1])/2
    if x!=y:
        vr = (h-l)*1.0/(x-y)
    df.loc[df.index[i+N],'VR'] = vr
    

df.head(10)

,Uopen,Ulow,Uhigh,Uclose,c,ma,R,VR
0,52.15,51.93,53.25,52.20,-0.14,NaN,NaN,NaN
1,52.65,52.04,53.58,52.34,-0.51,NaN,NaN,NaN
2,52.76,51.85,53.30,52.85,0.24,52.554,-44.242424,1.972414
3,52.91,52.29,53.25,52.61,-0.16,52.568,-56.069364,3.114583
4,52.17,51.82,52.95,52.77,0.50,52.726,-36.551724,2.132743
5,53.10,51.44,53.16,52.27,-0.86,52.804,-68.531469,1.215116
6,52.93,52.75,53.41,53.13,-0.11,52.878,-1.744186,4.318182
7,53.13,53.05,53.79,53.24,0.26,52.796,-8.629442,3.216216
8,52.00,51.76,53.13,52.98,0.62,52.782,-77.884615,1.021898
9,52.20,51.83,52.61,52.36,0.16,52.628,-70.443350,2.705128


In [7]:
df['RSI'] = ta.RSI(df['Uclose'].values, timeperiod = N)

df.head(10)

,Uopen,Ulow,Uhigh,Uclose,c,ma,R,VR,RSI
0,52.15,51.93,53.25,52.20,-0.14,NaN,NaN,NaN,NaN
1,52.65,52.04,53.58,52.34,-0.51,NaN,NaN,NaN,NaN
2,52.76,51.85,53.30,52.85,0.24,52.554,-44.242424,1.972414,100.000000
3,52.91,52.29,53.25,52.61,-0.16,52.568,-56.069364,3.114583,57.522124
4,52.17,51.82,52.95,52.77,0.50,52.726,-36.551724,2.132743,72.881356
5,53.10,51.44,53.16,52.27,-0.86,52.804,-68.531469,1.215116,22.357019
6,52.93,52.75,53.41,53.13,-0.11,52.878,-1.744186,4.318182,77.060932
7,53.13,53.05,53.79,53.24,0.26,52.796,-8.629442,3.216216,80.563991
8,52.00,51.76,53.13,52.98,0.62,52.782,-77.884615,1.021898,46.787604
9,52.20,51.83,52.61,52.36,0.16,52.628,-70.443350,2.705128,15.598488


In [8]:
df['EMA'] = df.iloc[:,3].ewm(span=N,adjust=False).mean()

df.head()

,Uopen,Ulow,Uhigh,Uclose,c,ma,R,VR,RSI,EMA
0,52.15,51.93,53.25,52.20,-0.14,NaN,NaN,NaN,NaN,52.200000
1,52.65,52.04,53.58,52.34,-0.51,NaN,NaN,NaN,NaN,52.293333
2,52.76,51.85,53.30,52.85,0.24,52.554,-44.242424,1.972414,100.000000,52.664444
3,52.91,52.29,53.25,52.61,-0.16,52.568,-56.069364,3.114583,57.522124,52.628148
4,52.17,51.82,52.95,52.77,0.50,52.726,-36.551724,2.132743,72.881356,52.722716


In [9]:
M = pd.Series(df['Uclose'].diff(1))  
aM = abs(M) 
r = N+1
s = N+1
EMA1 = M.ewm(span = r, min_periods = r - 1).mean()  
aEMA1 = aM.ewm(span = r, min_periods = r - 1).mean()
EMA2 = EMA1.ewm(span = s, min_periods = s - 1).mean() 
aEMA2 = aEMA1.ewm(span = s, min_periods = s - 1).mean()
TSI = pd.Series(EMA2 / aEMA2, name = 'TSI_' + str(r) + '_' + str(s))  
df['TSI'] = TSI

df.head(10)

,Uopen,Ulow,Uhigh,Uclose,c,ma,R,VR,RSI,EMA,TSI
0,52.15,51.93,53.25,52.20,-0.14,NaN,NaN,NaN,NaN,52.200000,NaN
1,52.65,52.04,53.58,52.34,-0.51,NaN,NaN,NaN,NaN,52.293333,NaN
2,52.76,51.85,53.30,52.85,0.24,52.554,-44.242424,1.972414,100.000000,52.664444,NaN
3,52.91,52.29,53.25,52.61,-0.16,52.568,-56.069364,3.114583,57.522124,52.628148,0.447217
4,52.17,51.82,52.95,52.77,0.50,52.726,-36.551724,2.132743,72.881356,52.722716,0.441525
5,53.10,51.44,53.16,52.27,-0.86,52.804,-68.531469,1.215116,22.357019,52.420905,-0.174366
6,52.93,52.75,53.41,53.13,-0.11,52.878,-1.744186,4.318182,77.060932,52.893635,0.304862
7,53.13,53.05,53.79,53.24,0.26,52.796,-8.629442,3.216216,80.563991,53.124545,0.439233
8,52.00,51.76,53.13,52.98,0.62,52.782,-77.884615,1.021898,46.787604,53.028182,0.221660
9,52.20,51.83,52.61,52.36,0.16,52.628,-70.443350,2.705128,15.598488,52.582727,-0.291408


In [10]:
for i in range(0,df.shape[0]-N-1):
    bs = ''
    for j in range(N+1):
        if df['c'][i+j]>=0:
            bs+= '0'
        else:
            bs+= '1' 
    bs = int(bs, 2)
    df.loc[df.index[i+N],'y'] = bs

df.head(10)

,Uopen,Ulow,Uhigh,Uclose,c,ma,R,VR,RSI,EMA,TSI,y
0,52.15,51.93,53.25,52.20,-0.14,NaN,NaN,NaN,NaN,52.200000,NaN,NaN
1,52.65,52.04,53.58,52.34,-0.51,NaN,NaN,NaN,NaN,52.293333,NaN,NaN
2,52.76,51.85,53.30,52.85,0.24,52.554,-44.242424,1.972414,100.000000,52.664444,NaN,6.0
3,52.91,52.29,53.25,52.61,-0.16,52.568,-56.069364,3.114583,57.522124,52.628148,0.447217,5.0
4,52.17,51.82,52.95,52.77,0.50,52.726,-36.551724,2.132743,72.881356,52.722716,0.441525,2.0
5,53.10,51.44,53.16,52.27,-0.86,52.804,-68.531469,1.215116,22.357019,52.420905,-0.174366,5.0
6,52.93,52.75,53.41,53.13,-0.11,52.878,-1.744186,4.318182,77.060932,52.893635,0.304862,3.0
7,53.13,53.05,53.79,53.24,0.26,52.796,-8.629442,3.216216,80.563991,53.124545,0.439233,6.0
8,52.00,51.76,53.13,52.98,0.62,52.782,-77.884615,1.021898,46.787604,53.028182,0.221660,4.0
9,52.20,51.83,52.61,52.36,0.16,52.628,-70.443350,2.705128,15.598488,52.582727,-0.291408,0.0


In [11]:
df = df.iloc[3:]
df = df.iloc[:2598]


In [12]:
X = df.iloc[:, 5:-1]
y = df.iloc[:, -1]
split = 2200
X_train, X_test, y_train, y_test = X[:split], X[split:], y[:split], y[split:]

# sc = StandardScaler()
# X_train = sc.fit_transform(X_train)
# X_test = sc.transform(X_test)

clf = svm.SVC(kernel = 'poly')
clf.fit(X_train, y_train)
confidence = clf.score(X_test, y_test)
print('accuracy:',confidence)
predictions = clf.predict(X_test)
print('predicted class counts:',Counter(predictions))

# confidence = 0
# clf = svm.SVC(kernel = 'poly')
# for i in range(2594):
#     split = i+3
#     X_train, X_test, y_train, y_test = X[i:split], X[split:split+1], y[i:split], y[split:split+1]
#     clf.fit(X_train, y_train)
#     confidence += clf.score(X_test, y_test)
#     predictions = clf.predict(X_test)
    
# print(confidence/2594)

accuracy: 0.31909547738693467
predicted class counts: Counter({2.0: 134, 1.0: 101, 4.0: 58, 7.0: 45, 5.0: 24, 6.0: 24, 0.0: 12})
